<center><h1>Amazon Web Scrapping</h1></center>

<h4>1) Importing the libraries</h4>



In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import seaborn as srn
import matplotlib.pyplot as plt

<h4>2) Setting the path of Chrome's driver</h4>

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
Driver has been 

<h4>3) Getting our search parameters</h4>

In [3]:
search_term = input('Write the product name')
search_term = search_term.replace(' ','+')  

The user will decide which product the program will scrap data

<h4>4) Iterating between products and pages</h4>

A empty list is created to storage our product data throughout the iterations

In [4]:
list_products = []

The next block of code will navigate between the first page to the last one, and the URL will get the three variables that we defined for our search.

The program will get the source code of the URL for each page (first for loop). The products are inside a tag div with a specific name, the program will iterate through the products (second for loop) and scrap five informations:
1) Product Name
2) Price
3) Rating
4) Review Count
5) Link

(I'm not considering sponsored products)<br>
If it returns AtributeError, the variable will receive a NULL value. The result will be a tuple with our 5 variables and after each product, this tuple will be storaged in the variable 'list_products' which is the empty list created right above.

In [5]:
page = 0
for page in range(1,8):
    url = f'https://www.amazon.com.br/s?k={search_term}&page={page}&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=32CUIJ6TTV5WT&qid=1646248377&sprefix={search_term}%2Caps%2C275&ref=sr_pg_{page}'    

    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})
    for product in results:
        sponsored_product = product.find('span','a-color-base').text.strip()
        if sponsored_product == "Patrocinados":
            continue
        
        a_tag = product.h2.a
        product_name = a_tag.text.strip()
        link = 'https://www.amazon.com.br' + a_tag.get('href')
        try:
            price_span_tag = product.find('span', 'a-price')
            price = price_span_tag.find('span', 'a-offscreen').text.strip()
        except AttributeError:
            price = ''
                
        try:
            rating_icon = product.find('span','a-icon-alt') .text.strip()
            review_count = product.find('span', {'class': 'a-size-base s-underline-text'}).text
            review_count = str(review_count)
            review_count = review_count.replace('.','')
        except AttributeError:
            rating = ''
            review_count = ''

        result = (product_name, price, rating_icon, review_count, link)
        list_products.append(result)
driver.close()

I previously created a csv file, whose will get the products (list_products) and right after will be transformed in a pandas dataframe

In [6]:
with open('products.csv','w', encoding = 'utf-8') as p:
    writer = csv.writer(p)
    writer.writerow(['PRODUCT DESCRIPTION','PRICE','RATING','REVIEW COUNT','LINK'])
    writer.writerows(list_products)

In [7]:
df = pd.read_csv('products.csv')
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Headphone Philips Wireless TAUH202BK/00 - Pret...,"R$ 99,00","4,6 de 5 estrelas",5839.0,https://www.amazon.com.br/Ouvido-Philips-Wirel...
1,Headphone Bluetooth Joy P2 Preto Multilaser – ...,"R$ 89,10","4,3 de 5 estrelas",2022.0,https://www.amazon.com.br/Headphone-Bluetooth-...
2,Mpow 059 Lite Fone Ouvido Bluetooth 5.0 Preto ...,"R$ 132,18","4,9 de 5 estrelas",18.0,https://www.amazon.com.br/Bluetooth-Bateria-Mi...
3,Headphone Bluetooth Comfort Go i2GO Com Microf...,"R$ 159,00","4,3 de 5 estrelas",1640.0,https://www.amazon.com.br/Headphone-Bluetooth-...
4,Headphone Philips bluetooth over-ear com micro...,"R$ 209,78","3,9 de 5 estrelas",321.0,https://www.amazon.com.br/Headphone-bluetooth-...


<h4>5) Doing some EDA (Exploratory Data Analysis)</h4>

Checking the type of each column

In [8]:
print(df.dtypes)

PRODUCT DESCRIPTION     object
PRICE                   object
RATING                  object
REVIEW COUNT           float64
LINK                    object
dtype: object


Checking the number of null values in each column

In [9]:
df.isnull().sum()

PRODUCT DESCRIPTION     0
PRICE                  20
RATING                  0
REVIEW COUNT           27
LINK                    0
dtype: int64

Removing the NAn

In [10]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

PRODUCT DESCRIPTION    0
PRICE                  0
RATING                 0
REVIEW COUNT           0
LINK                   0
dtype: int64

Treating the 'RATING' column to get only the number

In [11]:
df['RATING'] = df['RATING'].str[:3].replace({',':'.'}, regex=True).astype(float)
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Headphone Philips Wireless TAUH202BK/00 - Pret...,"R$ 99,00",4.6,5839.0,https://www.amazon.com.br/Ouvido-Philips-Wirel...
1,Headphone Bluetooth Joy P2 Preto Multilaser – ...,"R$ 89,10",4.3,2022.0,https://www.amazon.com.br/Headphone-Bluetooth-...
2,Mpow 059 Lite Fone Ouvido Bluetooth 5.0 Preto ...,"R$ 132,18",4.9,18.0,https://www.amazon.com.br/Bluetooth-Bateria-Mi...
3,Headphone Bluetooth Comfort Go i2GO Com Microf...,"R$ 159,00",4.3,1640.0,https://www.amazon.com.br/Headphone-Bluetooth-...
4,Headphone Philips bluetooth over-ear com micro...,"R$ 209,78",3.9,321.0,https://www.amazon.com.br/Headphone-bluetooth-...


The column 'REVIEW COUNT' will get only integer values

In [12]:
#df['REVIEW COUNT'] = df['REVIEW COUNT'].replace({'.':''}, regex=True).astype(int)
df['REVIEW COUNT'] = df['REVIEW COUNT'].astype('int')
#df['REVIEW COUNT'] = df['REVIEW COUNT'].str.replace({'.':','},regex=True)
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Headphone Philips Wireless TAUH202BK/00 - Pret...,"R$ 99,00",4.6,5839,https://www.amazon.com.br/Ouvido-Philips-Wirel...
1,Headphone Bluetooth Joy P2 Preto Multilaser – ...,"R$ 89,10",4.3,2022,https://www.amazon.com.br/Headphone-Bluetooth-...
2,Mpow 059 Lite Fone Ouvido Bluetooth 5.0 Preto ...,"R$ 132,18",4.9,18,https://www.amazon.com.br/Bluetooth-Bateria-Mi...
3,Headphone Bluetooth Comfort Go i2GO Com Microf...,"R$ 159,00",4.3,1640,https://www.amazon.com.br/Headphone-Bluetooth-...
4,Headphone Philips bluetooth over-ear com micro...,"R$ 209,78",3.9,321,https://www.amazon.com.br/Headphone-bluetooth-...


In [13]:
df.dtypes

PRODUCT DESCRIPTION     object
PRICE                   object
RATING                 float64
REVIEW COUNT             int32
LINK                    object
dtype: object

Removing the 'R$' part from the price

In [14]:
remove_coin = df['PRICE'].str.split('$').str[1]
df['PRICE'] = remove_coin.str.strip()
#price_integer = 


To correct the column 'Price' i had to split the numbers in two parts (integer part and decimal part) and replace ',' to a empty value. Then I had to put these two parts together again with '.' to separate them. It's a not so pretty solution but it worked, and I intend to optimize this part.

<h5>Splitting and Replacing</h5>

In [15]:

df['integer_part'] = df['PRICE'].str.split(',').str[0]
df['integer_part'] = df['integer_part'].str.replace('.','')
df['integer_part']

C:\Users\Alfred\AppData\Local\Temp\ipykernel_48604\1546184746.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['integer_part'] = df['integer_part'].str.replace('.','')


0       99
1       89
2      132
3      159
4      209
      ... 
254    149
255    135
256    740
257    234
258    149
Name: integer_part, Length: 259, dtype: object

In [16]:
df['decimal_part'] = df['PRICE'].str.split(',').str[1]
df['decimal_part']

0      00
1      10
2      18
3      00
4      78
       ..
254    90
255    42
256    95
257    90
258    98
Name: decimal_part, Length: 259, dtype: object

<h5>Joining the parts</h5>

In [17]:
df['PRICE_CORRECT'] = df['integer_part'] + "." + df['decimal_part']
df['PRICE_CORRECT'] = df['PRICE_CORRECT'].astype('string')
df['PRICE_CORRECT'] = df['PRICE_CORRECT'].astype('float')

In [18]:
df['PRICE'] = df['PRICE_CORRECT']
df.drop(['integer_part','decimal_part','PRICE_CORRECT'], axis=1, inplace=True)


Now that the dataframe is polished, let's check how the variables 'Rating' and 'Price' are distributed. It's possible to see that the most part of Power Banks in Amazon have a sort of good quality, due to de high rating. And the Average price is R$100,00 with some outliers. 

In [33]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [36]:
fig1 = px.histogram(df['RATING'], color_discrete_sequence=['#520AA9'], title='RATING DISTRIBUTION', width=800, height=350)
fig1.show()

In [37]:
fig2 = px.histogram(df['PRICE'], color_discrete_sequence=['#520AA9'], title='PRICE DISTRIBUTION', width=800, height=350)
fig2.show()

In [38]:
fig3 = px.histogram(df['REVIEW COUNT'], color_discrete_sequence=['#520AA9'], title='REVIEWS DISTRIBUTION', width=800, height=350)
fig3.show()

In [39]:
df.head()

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
0,Headphone Philips Wireless TAUH202BK/00 - Pret...,99.00,4.6,5839,https://www.amazon.com.br/Ouvido-Philips-Wirel...
1,Headphone Bluetooth Joy P2 Preto Multilaser – ...,89.10,4.3,2022,https://www.amazon.com.br/Headphone-Bluetooth-...
2,Mpow 059 Lite Fone Ouvido Bluetooth 5.0 Preto ...,132.18,4.9,18,https://www.amazon.com.br/Bluetooth-Bateria-Mi...
3,Headphone Bluetooth Comfort Go i2GO Com Microf...,159.00,4.3,1640,https://www.amazon.com.br/Headphone-Bluetooth-...
4,Headphone Philips bluetooth over-ear com micro...,209.78,3.9,321,https://www.amazon.com.br/Headphone-bluetooth-...


Organizing the data to show the most reviwed and most rated products first

In [40]:
df.sort_values(by=['REVIEW COUNT','RATING'], inplace=True, ascending=False)
df.head(10)

,PRODUCT DESCRIPTION,PRICE,RATING,REVIEW COUNT,LINK
155,TOZO T10 Bluetooth 5.0 Wireless Earbuds with W...,229.90,4.3,175326,https://www.amazon.com.br/TOZO-Bluetooth-carre...
125,TOZO T6 True Wireless Earbuds Bluetooth Headph...,281.90,4.4,169079,https://www.amazon.com.br/TOZO-T6-True-carrega...
143,Fones de ouvido iJoy Fosco recarregável sem fi...,420.00,4.4,64854,https://www.amazon.com.br/ouvido-recarreg%C3%A...
51,"Fones de ouvido Bluetooth, fones de ouvido Zih...",180.99,4.4,48373,https://www.amazon.com.br/Bluetooth-Over-Ear-e...
19,Fone de ouvido Beats Solo3 Wireless - The Beat...,974.14,4.7,41028,https://www.amazon.com.br/Beats-Solo3-fones-ou...
37,Sony WI-XB400 Fone de ouvido intra-auricular s...,299.00,4.2,35975,https://www.amazon.com.br/ouvido-Sony-Wi-Xb400...
30,Fone de Ouvido Bluetooth JBL Tune 500 On Ear P...,215.00,4.7,35100,https://www.amazon.com.br/Fone-Ouvido-JBL-Tune...
183,TOZO T12 Wireless Earbuds Bluetooth Headphones...,329.00,4.3,31651,https://www.amazon.com.br/TOZO-T12-Headphones-...
119,Headphone Sony WH-1000XM4 Preto sem fio Blueto...,1960.00,4.7,22468,https://www.amazon.com.br/Sony-WH-1000XM4-Canc...
112,TOZO NC9 Hybrid Active Noise Cancelling Wirele...,394.95,4.2,21351,https://www.amazon.com.br/TOZO-Cancelling-Head...
